### =====================================================================
### IMPORTACIÓN GENERAL DE LA INFORMACIÓN.
### =====================================================================

In [1]:
## IMPORTACIÓN GENERAL DE LIBRERIAS Y VISUALIZACIÓN DE DATOS (matplotlib y seaborn)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as DT
import warnings
import descartes
import geopandas as gpd
import json
import requests
import geocoder

from sklearn.ensemble import RandomForestRegressor
from shapely.geometry import Point, Polygon
from urllib2 import urlopen

%matplotlib inline
warnings.filterwarnings('ignore')
plt.style.use('default') 
sns.set(style="whitegrid") 
plt.rcParams['figure.figsize'] = (15, 10)

In [2]:
#lEEMOS EL CSV
train = pd.read_csv('../../DATA_TP2/train.csv')
test = pd.read_csv('../../DATA_TP2/test.csv')
respuesta = pd.read_csv('../../DATA_TP2/ejemploRespuesta.csv')

### =====================================================================
### PREPARACIÓN DE DATOS, FEATURE ENGINEERING.
### =====================================================================

In [3]:
train['excelente'] = 0
train.loc[train.descripcion.str.contains('vigilancia', na=False), 'vigilancia'] = 1
train['vigilancia'] = 0
train.loc[train.descripcion.str.contains('excelente', na=False), 'excelente'] = 1
train['esquina'] = 0
train.loc[train.descripcion.str.contains('esquina', na=False), 'esquina'] = 1
train['remodelada'] = 0
train.loc[train.descripcion.str.contains('remodelada', na=False), 'remodelada'] = 1
train['lujo'] = 0
train.loc[train.descripcion.str.contains('lujo', na=False), 'lujo'] = 1

In [4]:
train['lat'].fillna(0, inplace=True)
train['lng'].fillna(0, inplace=True)

In [5]:
train['antiguedad_agrupada_0'] = 0
train['antiguedad_agrupada_10'] = 0
train['antiguedad_agrupada_20'] = 0
train['antiguedad_agrupada_30'] = 0
train['antiguedad_agrupada_40'] = 0
train['antiguedad_agrupada_50'] = 0
train['antiguedad_agrupada_60'] = 0
train['antiguedad_agrupada_70'] = 0
train['antiguedad_agrupada_80'] = 0
train['antiguedad_agrupada_90'] = 0
train['antiguedad_agrupada_100'] = 0

train.loc[train.antiguedad <= 5, 'antiguedad_agrupada_0'] = 1
train.loc[train.antiguedad > 95, 'antiguedad_agrupada_100'] = 1
train.loc[((train.antiguedad > 5) & (train.antiguedad <= 15)), 'antiguedad_agrupada_10'] = 1
train.loc[((train.antiguedad > 15) & (train.antiguedad <= 25)), 'antiguedad_agrupada_20'] = 1
train.loc[((train.antiguedad > 25) & (train.antiguedad <= 35)), 'antiguedad_agrupada_30'] = 1
train.loc[((train.antiguedad > 35) & (train.antiguedad <= 45)), 'antiguedad_agrupada_40'] = 1
train.loc[((train.antiguedad > 45) & (train.antiguedad <= 55)), 'antiguedad_agrupada_50'] = 1
train.loc[((train.antiguedad > 55) & (train.antiguedad <= 65)), 'antiguedad_agrupada_60'] = 1
train.loc[((train.antiguedad > 65) & (train.antiguedad <= 75)), 'antiguedad_agrupada_70'] = 1
train.loc[((train.antiguedad > 75) & (train.antiguedad <= 85)), 'antiguedad_agrupada_80'] = 1
train.loc[((train.antiguedad > 85) & (train.antiguedad <= 95)), 'antiguedad_agrupada_90'] = 1

In [6]:
train['metroscubiertos_agrupada_0'] = 0
train['metroscubiertos_agrupada_10'] = 0
train['metroscubiertos_agrupada_20'] = 0
train['metroscubiertos_agrupada_30'] = 0
train['metroscubiertos_agrupada_40'] = 0
train['metroscubiertos_agrupada_50'] = 0
train['metroscubiertos_agrupada_60'] = 0
train['metroscubiertos_agrupada_70'] = 0
train['metroscubiertos_agrupada_80'] = 0
train['metroscubiertos_agrupada_90'] = 0
train['metroscubiertos_agrupada_100'] = 0
train['metroscubiertos_agrupada_110'] = 0
train['metroscubiertos_agrupada_125'] = 0
train['metroscubiertos_agrupada_150'] = 0
train['metroscubiertos_agrupada_200'] = 0
train['metroscubiertos_agrupada_250'] = 0
train['metroscubiertos_agrupada_300'] = 0
train['metroscubiertos_agrupada_350'] = 0
train['metroscubiertos_agrupada_400'] = 0
train['metroscubiertos_agrupada_450'] = 0
train['metroscubiertos_agrupada_500'] = 0
train['metroscubiertos_agrupada_550'] = 0
train['metroscubiertos_agrupada_600'] = 0
train['metroscubiertos_agrupada_650'] = 0
train['metroscubiertos_agrupada_700'] = 0
train['metroscubiertos_agrupada_750'] = 0
train['metroscubiertos_agrupada_800'] = 0
train['metroscubiertos_agrupada_1000'] = 0

train.loc[train.metroscubiertos <= 5, 'metroscubiertos_agrupada_0'] = 1
train.loc[train.metroscubiertos > 805, 'metroscubiertos_agrupada_1000'] = 1
train.loc[((train.metroscubiertos > 5) & (train.metroscubiertos <= 15)), 'metroscubiertos_agrupada_10'] = 1
train.loc[((train.metroscubiertos > 15) & (train.metroscubiertos <= 25)), 'metroscubiertos_agrupada_20'] = 1
train.loc[((train.metroscubiertos > 25) & (train.metroscubiertos <= 35)), 'metroscubiertos_agrupada_30'] = 1
train.loc[((train.metroscubiertos > 35) & (train.metroscubiertos <= 45)), 'metroscubiertos_agrupada_40'] = 1
train.loc[((train.metroscubiertos > 45) & (train.metroscubiertos <= 55)), 'metroscubiertos_agrupada_50'] = 1
train.loc[((train.metroscubiertos > 55) & (train.metroscubiertos <= 65)), 'metroscubiertos_agrupada_60'] = 1
train.loc[((train.metroscubiertos > 65) & (train.metroscubiertos <= 75)), 'metroscubiertos_agrupada_70'] = 1
train.loc[((train.metroscubiertos > 75) & (train.metroscubiertos <= 85)), 'metroscubiertos_agrupada_80'] = 1
train.loc[((train.metroscubiertos > 85) & (train.metroscubiertos <= 95)), 'metroscubiertos_agrupada_90'] = 1
train.loc[((train.metroscubiertos > 95) & (train.metroscubiertos <= 105)), 'metroscubiertos_agrupada_100'] = 1
train.loc[((train.metroscubiertos > 105) & (train.metroscubiertos <= 115)), 'metroscubiertos_agrupada_110'] = 1
train.loc[((train.metroscubiertos > 115) & (train.metroscubiertos <= 130)), 'metroscubiertos_agrupada_125'] = 1
train.loc[((train.metroscubiertos > 130) & (train.metroscubiertos <= 155)), 'metroscubiertos_agrupada_150'] = 1
train.loc[((train.metroscubiertos > 155) & (train.metroscubiertos <= 205)), 'metroscubiertos_agrupada_200'] = 1
train.loc[((train.metroscubiertos > 205) & (train.metroscubiertos <= 255)), 'metroscubiertos_agrupada_250'] = 1
train.loc[((train.metroscubiertos > 255) & (train.metroscubiertos <= 305)), 'metroscubiertos_agrupada_300'] = 1
train.loc[((train.metroscubiertos > 305) & (train.metroscubiertos <= 355)), 'metroscubiertos_agrupada_350'] = 1
train.loc[((train.metroscubiertos > 355) & (train.metroscubiertos <= 405)), 'metroscubiertos_agrupada_400'] = 1
train.loc[((train.metroscubiertos > 405) & (train.metroscubiertos <= 455)), 'metroscubiertos_agrupada_450'] = 1
train.loc[((train.metroscubiertos > 455) & (train.metroscubiertos <= 500)), 'metroscubiertos_agrupada_500'] = 1
train.loc[((train.metroscubiertos > 505) & (train.metroscubiertos <= 555)), 'metroscubiertos_agrupada_550'] = 1
train.loc[((train.metroscubiertos > 555) & (train.metroscubiertos <= 600)), 'metroscubiertos_agrupada_600'] = 1
train.loc[((train.metroscubiertos > 605) & (train.metroscubiertos <= 655)), 'metroscubiertos_agrupada_650'] = 1
train.loc[((train.metroscubiertos > 655) & (train.metroscubiertos <= 700)), 'metroscubiertos_agrupada_700'] = 1
train.loc[((train.metroscubiertos > 705) & (train.metroscubiertos <= 755)), 'metroscubiertos_agrupada_750'] = 1
train.loc[((train.metroscubiertos > 755) & (train.metroscubiertos <= 805)), 'metroscubiertos_agrupada_800'] = 1

In [7]:
train['metrostotales_agrupada_0'] = 0
train['metrostotales_agrupada_10'] = 0
train['metrostotales_agrupada_20'] = 0
train['metrostotales_agrupada_30'] = 0
train['metrostotales_agrupada_40'] = 0
train['metrostotales_agrupada_50'] = 0
train['metrostotales_agrupada_60'] = 0
train['metrostotales_agrupada_70'] = 0
train['metrostotales_agrupada_80'] = 0
train['metrostotales_agrupada_90'] = 0
train['metrostotales_agrupada_100'] = 0
train['metrostotales_agrupada_110'] = 0
train['metrostotales_agrupada_125'] = 0
train['metrostotales_agrupada_150'] = 0
train['metrostotales_agrupada_200'] = 0
train['metrostotales_agrupada_250'] = 0
train['metrostotales_agrupada_300'] = 0
train['metrostotales_agrupada_350'] = 0
train['metrostotales_agrupada_400'] = 0
train['metrostotales_agrupada_450'] = 0
train['metrostotales_agrupada_500'] = 0
train['metrostotales_agrupada_550'] = 0
train['metrostotales_agrupada_600'] = 0
train['metrostotales_agrupada_650'] = 0
train['metrostotales_agrupada_700'] = 0
train['metrostotales_agrupada_750'] = 0
train['metrostotales_agrupada_800'] = 0
train['metrostotales_agrupada_1000'] = 0

train.loc[train.metrostotales <= 5, 'metrostotales_agrupada_0'] = 1
train.loc[train.metrostotales > 805, 'metrostotales_agrupada_1000'] = 1
train.loc[((train.metrostotales > 5) & (train.metrostotales <= 15)), 'metrostotales_agrupada_10'] = 1
train.loc[((train.metrostotales > 15) & (train.metrostotales <= 25)), 'metrostotales_agrupada_20'] = 1
train.loc[((train.metrostotales > 25) & (train.metrostotales <= 35)), 'metrostotales_agrupada_30'] = 1
train.loc[((train.metrostotales > 35) & (train.metrostotales <= 45)), 'metrostotales_agrupada_40'] = 1
train.loc[((train.metrostotales > 45) & (train.metrostotales <= 55)), 'metrostotales_agrupada_50'] = 1
train.loc[((train.metrostotales > 55) & (train.metrostotales <= 65)), 'metrostotales_agrupada_60'] = 1
train.loc[((train.metrostotales > 65) & (train.metrostotales <= 75)), 'metrostotales_agrupada_70'] = 1
train.loc[((train.metrostotales > 75) & (train.metrostotales <= 85)), 'metrostotales_agrupada_80'] = 1
train.loc[((train.metrostotales > 85) & (train.metrostotales <= 95)), 'metrostotales_agrupada_90'] = 1
train.loc[((train.metrostotales > 95) & (train.metrostotales <= 105)), 'metrostotales_agrupada_100'] = 1
train.loc[((train.metrostotales > 105) & (train.metrostotales <= 115)), 'metrostotales_agrupada_110'] = 1
train.loc[((train.metrostotales > 115) & (train.metrostotales <= 130)), 'metrostotales_agrupada_125'] = 1
train.loc[((train.metrostotales > 130) & (train.metrostotales <= 155)), 'metrostotales_agrupada_150'] = 1
train.loc[((train.metrostotales > 155) & (train.metrostotales <= 205)), 'metrostotales_agrupada_200'] = 1
train.loc[((train.metrostotales > 205) & (train.metrostotales <= 255)), 'metrostotales_agrupada_250'] = 1
train.loc[((train.metrostotales > 255) & (train.metrostotales <= 305)), 'metrostotales_agrupada_300'] = 1
train.loc[((train.metrostotales > 305) & (train.metrostotales <= 355)), 'metrostotales_agrupada_350'] = 1
train.loc[((train.metrostotales > 355) & (train.metrostotales <= 405)), 'metrostotales_agrupada_400'] = 1
train.loc[((train.metrostotales > 405) & (train.metrostotales <= 455)), 'metrostotales_agrupada_450'] = 1
train.loc[((train.metrostotales > 455) & (train.metrostotales <= 500)), 'metrostotales_agrupada_500'] = 1
train.loc[((train.metrostotales > 505) & (train.metrostotales <= 555)), 'metrostotales_agrupada_550'] = 1
train.loc[((train.metrostotales > 555) & (train.metrostotales <= 600)), 'metrostotales_agrupada_600'] = 1
train.loc[((train.metrostotales > 605) & (train.metrostotales <= 655)), 'metrostotales_agrupada_650'] = 1
train.loc[((train.metrostotales > 655) & (train.metrostotales <= 700)), 'metrostotales_agrupada_700'] = 1
train.loc[((train.metrostotales > 705) & (train.metrostotales <= 755)), 'metrostotales_agrupada_750'] = 1
train.loc[((train.metrostotales > 755) & (train.metrostotales <= 805)), 'metrostotales_agrupada_800'] = 1

In [8]:
train['fecha'] = pd.to_datetime(train['fecha'], infer_datetime_format=True)
train['mes'] = train['fecha'].dt.month
train['dia'] = train['fecha'].dt.dayofweek
train['anio'] = train['fecha'].dt.year

In [9]:
train.loc[train.tipodepropiedad.str.contains('Comercial', na=False), 'tipodepropiedad'] = 'Comercio'
train.loc[train.tipodepropiedad.str.contains('Casa en condominio', na=False), 'tipodepropiedad'] = 'Condominio'

### =====================================================================
### TRATAMIENTO SOBRE TEST.
### =====================================================================

In [10]:
test['excelente'] = 0
test.loc[test.descripcion.str.contains('vigilancia', na=False), 'vigilancia'] = 1
test['vigilancia'] = 0
test.loc[test.descripcion.str.contains('excelente', na=False), 'excelente'] = 1
test['esquina'] = 0
test.loc[test.descripcion.str.contains('esquina', na=False), 'esquina'] = 1
test['remodelada'] = 0
test.loc[test.descripcion.str.contains('remodelada', na=False), 'remodelada'] = 1
test['lujo'] = 0
test.loc[test.descripcion.str.contains('lujo', na=False), 'lujo'] = 1

In [11]:
test['lat'].fillna(0, inplace=True)
test['lng'].fillna(0, inplace=True)

In [12]:
test['antiguedad_agrupada_0'] = 0
test['antiguedad_agrupada_10'] = 0
test['antiguedad_agrupada_20'] = 0
test['antiguedad_agrupada_30'] = 0
test['antiguedad_agrupada_40'] = 0
test['antiguedad_agrupada_50'] = 0
test['antiguedad_agrupada_60'] = 0
test['antiguedad_agrupada_70'] = 0
test['antiguedad_agrupada_80'] = 0
test['antiguedad_agrupada_90'] = 0
test['antiguedad_agrupada_100'] = 0

test.loc[test.antiguedad <= 5, 'antiguedad_agrupada_0'] = 1
test.loc[test.antiguedad > 95, 'antiguedad_agrupada_100'] = 1
test.loc[((test.antiguedad > 5) & (test.antiguedad <= 15)), 'antiguedad_agrupada_10'] = 1
test.loc[((test.antiguedad > 15) & (test.antiguedad <= 25)), 'antiguedad_agrupada_20'] = 1
test.loc[((test.antiguedad > 25) & (test.antiguedad <= 35)), 'antiguedad_agrupada_30'] = 1
test.loc[((test.antiguedad > 35) & (test.antiguedad <= 45)), 'antiguedad_agrupada_40'] = 1
test.loc[((test.antiguedad > 45) & (test.antiguedad <= 55)), 'antiguedad_agrupada_50'] = 1
test.loc[((test.antiguedad > 55) & (test.antiguedad <= 65)), 'antiguedad_agrupada_60'] = 1
test.loc[((test.antiguedad > 65) & (test.antiguedad <= 75)), 'antiguedad_agrupada_70'] = 1
test.loc[((test.antiguedad > 75) & (test.antiguedad <= 85)), 'antiguedad_agrupada_80'] = 1
test.loc[((test.antiguedad > 85) & (test.antiguedad <= 95)), 'antiguedad_agrupada_90'] = 1

In [13]:
test['metroscubiertos_agrupada_0'] = 0
test['metroscubiertos_agrupada_10'] = 0
test['metroscubiertos_agrupada_20'] = 0
test['metroscubiertos_agrupada_30'] = 0
test['metroscubiertos_agrupada_40'] = 0
test['metroscubiertos_agrupada_50'] = 0
test['metroscubiertos_agrupada_60'] = 0
test['metroscubiertos_agrupada_70'] = 0
test['metroscubiertos_agrupada_80'] = 0
test['metroscubiertos_agrupada_90'] = 0
test['metroscubiertos_agrupada_100'] = 0
test['metroscubiertos_agrupada_110'] = 0
test['metroscubiertos_agrupada_125'] = 0
test['metroscubiertos_agrupada_150'] = 0
test['metroscubiertos_agrupada_200'] = 0
test['metroscubiertos_agrupada_250'] = 0
test['metroscubiertos_agrupada_300'] = 0
test['metroscubiertos_agrupada_350'] = 0
test['metroscubiertos_agrupada_400'] = 0
test['metroscubiertos_agrupada_450'] = 0
test['metroscubiertos_agrupada_500'] = 0
test['metroscubiertos_agrupada_550'] = 0
test['metroscubiertos_agrupada_600'] = 0
test['metroscubiertos_agrupada_650'] = 0
test['metroscubiertos_agrupada_700'] = 0
test['metroscubiertos_agrupada_750'] = 0
test['metroscubiertos_agrupada_800'] = 0
test['metroscubiertos_agrupada_1000'] = 0

test.loc[test.metroscubiertos <= 5, 'metroscubiertos_agrupada_0'] = 1
test.loc[test.metroscubiertos > 805, 'metroscubiertos_agrupada_1000'] = 1
test.loc[((test.metroscubiertos > 5) & (test.metroscubiertos <= 15)), 'metroscubiertos_agrupada_10'] = 1
test.loc[((test.metroscubiertos > 15) & (test.metroscubiertos <= 25)), 'metroscubiertos_agrupada_20'] = 1
test.loc[((test.metroscubiertos > 25) & (test.metroscubiertos <= 35)), 'metroscubiertos_agrupada_30'] = 1
test.loc[((test.metroscubiertos > 35) & (test.metroscubiertos <= 45)), 'metroscubiertos_agrupada_40'] = 1
test.loc[((test.metroscubiertos > 45) & (test.metroscubiertos <= 55)), 'metroscubiertos_agrupada_50'] = 1
test.loc[((test.metroscubiertos > 55) & (test.metroscubiertos <= 65)), 'metroscubiertos_agrupada_60'] = 1
test.loc[((test.metroscubiertos > 65) & (test.metroscubiertos <= 75)), 'metroscubiertos_agrupada_70'] = 1
test.loc[((test.metroscubiertos > 75) & (test.metroscubiertos <= 85)), 'metroscubiertos_agrupada_80'] = 1
test.loc[((test.metroscubiertos > 85) & (test.metroscubiertos <= 95)), 'metroscubiertos_agrupada_90'] = 1
test.loc[((test.metroscubiertos > 95) & (test.metroscubiertos <= 105)), 'metroscubiertos_agrupada_100'] = 1
test.loc[((test.metroscubiertos > 105) & (test.metroscubiertos <= 115)), 'metroscubiertos_agrupada_110'] = 1
test.loc[((test.metroscubiertos > 115) & (test.metroscubiertos <= 130)), 'metroscubiertos_agrupada_125'] = 1
test.loc[((test.metroscubiertos > 130) & (test.metroscubiertos <= 155)), 'metroscubiertos_agrupada_150'] = 1
test.loc[((test.metroscubiertos > 155) & (test.metroscubiertos <= 205)), 'metroscubiertos_agrupada_200'] = 1
test.loc[((test.metroscubiertos > 205) & (test.metroscubiertos <= 255)), 'metroscubiertos_agrupada_250'] = 1
test.loc[((test.metroscubiertos > 255) & (test.metroscubiertos <= 305)), 'metroscubiertos_agrupada_300'] = 1
test.loc[((test.metroscubiertos > 305) & (test.metroscubiertos <= 355)), 'metroscubiertos_agrupada_350'] = 1
test.loc[((test.metroscubiertos > 355) & (test.metroscubiertos <= 405)), 'metroscubiertos_agrupada_400'] = 1
test.loc[((test.metroscubiertos > 405) & (test.metroscubiertos <= 455)), 'metroscubiertos_agrupada_450'] = 1
test.loc[((test.metroscubiertos > 455) & (test.metroscubiertos <= 500)), 'metroscubiertos_agrupada_500'] = 1
test.loc[((test.metroscubiertos > 505) & (test.metroscubiertos <= 555)), 'metroscubiertos_agrupada_550'] = 1
test.loc[((test.metroscubiertos > 555) & (test.metroscubiertos <= 600)), 'metroscubiertos_agrupada_600'] = 1
test.loc[((test.metroscubiertos > 605) & (test.metroscubiertos <= 655)), 'metroscubiertos_agrupada_650'] = 1
test.loc[((test.metroscubiertos > 655) & (test.metroscubiertos <= 700)), 'metroscubiertos_agrupada_700'] = 1
test.loc[((test.metroscubiertos > 705) & (test.metroscubiertos <= 755)), 'metroscubiertos_agrupada_750'] = 1
test.loc[((test.metroscubiertos > 755) & (test.metroscubiertos <= 805)), 'metroscubiertos_agrupada_800'] = 1

In [14]:
test['metrostotales_agrupada_0'] = 0
test['metrostotales_agrupada_10'] = 0
test['metrostotales_agrupada_20'] = 0
test['metrostotales_agrupada_30'] = 0
test['metrostotales_agrupada_40'] = 0
test['metrostotales_agrupada_50'] = 0
test['metrostotales_agrupada_60'] = 0
test['metrostotales_agrupada_70'] = 0
test['metrostotales_agrupada_80'] = 0
test['metrostotales_agrupada_90'] = 0
test['metrostotales_agrupada_100'] = 0
test['metrostotales_agrupada_110'] = 0
test['metrostotales_agrupada_125'] = 0
test['metrostotales_agrupada_150'] = 0
test['metrostotales_agrupada_200'] = 0
test['metrostotales_agrupada_250'] = 0
test['metrostotales_agrupada_300'] = 0
test['metrostotales_agrupada_350'] = 0
test['metrostotales_agrupada_400'] = 0
test['metrostotales_agrupada_450'] = 0
test['metrostotales_agrupada_500'] = 0
test['metrostotales_agrupada_550'] = 0
test['metrostotales_agrupada_600'] = 0
test['metrostotales_agrupada_650'] = 0
test['metrostotales_agrupada_700'] = 0
test['metrostotales_agrupada_750'] = 0
test['metrostotales_agrupada_800'] = 0
test['metrostotales_agrupada_1000'] = 0

test.loc[test.metrostotales <= 5, 'metrostotales_agrupada_0'] = 1
test.loc[test.metrostotales > 805, 'metrostotales_agrupada_1000'] = 1
test.loc[((test.metrostotales > 5) & (test.metrostotales <= 15)), 'metrostotales_agrupada_10'] = 1
test.loc[((test.metrostotales > 15) & (test.metrostotales <= 25)), 'metrostotales_agrupada_20'] = 1
test.loc[((test.metrostotales > 25) & (test.metrostotales <= 35)), 'metrostotales_agrupada_30'] = 1
test.loc[((test.metrostotales > 35) & (test.metrostotales <= 45)), 'metrostotales_agrupada_40'] = 1
test.loc[((test.metrostotales > 45) & (test.metrostotales <= 55)), 'metrostotales_agrupada_50'] = 1
test.loc[((test.metrostotales > 55) & (test.metrostotales <= 65)), 'metrostotales_agrupada_60'] = 1
test.loc[((test.metrostotales > 65) & (test.metrostotales <= 75)), 'metrostotales_agrupada_70'] = 1
test.loc[((test.metrostotales > 75) & (test.metrostotales <= 85)), 'metrostotales_agrupada_80'] = 1
test.loc[((test.metrostotales > 85) & (test.metrostotales <= 95)), 'metrostotales_agrupada_90'] = 1
test.loc[((test.metrostotales > 95) & (test.metrostotales <= 105)), 'metrostotales_agrupada_100'] = 1
test.loc[((test.metrostotales > 105) & (test.metrostotales <= 115)), 'metrostotales_agrupada_110'] = 1
test.loc[((test.metrostotales > 115) & (test.metrostotales <= 130)), 'metrostotales_agrupada_125'] = 1
test.loc[((test.metrostotales > 130) & (test.metrostotales <= 155)), 'metrostotales_agrupada_150'] = 1
test.loc[((test.metrostotales > 155) & (test.metrostotales <= 205)), 'metrostotales_agrupada_200'] = 1
test.loc[((test.metrostotales > 205) & (test.metrostotales <= 255)), 'metrostotales_agrupada_250'] = 1
test.loc[((test.metrostotales > 255) & (test.metrostotales <= 305)), 'metrostotales_agrupada_300'] = 1
test.loc[((test.metrostotales > 305) & (test.metrostotales <= 355)), 'metrostotales_agrupada_350'] = 1
test.loc[((test.metrostotales > 355) & (test.metrostotales <= 405)), 'metrostotales_agrupada_400'] = 1
test.loc[((test.metrostotales > 405) & (test.metrostotales <= 455)), 'metrostotales_agrupada_450'] = 1
test.loc[((test.metrostotales > 455) & (test.metrostotales <= 500)), 'metrostotales_agrupada_500'] = 1
test.loc[((test.metrostotales > 505) & (test.metrostotales <= 555)), 'metrostotales_agrupada_550'] = 1
test.loc[((test.metrostotales > 555) & (test.metrostotales <= 600)), 'metrostotales_agrupada_600'] = 1
test.loc[((test.metrostotales > 605) & (test.metrostotales <= 655)), 'metrostotales_agrupada_650'] = 1
test.loc[((test.metrostotales > 655) & (test.metrostotales <= 700)), 'metrostotales_agrupada_700'] = 1
test.loc[((test.metrostotales > 705) & (test.metrostotales <= 755)), 'metrostotales_agrupada_750'] = 1
test.loc[((test.metrostotales > 755) & (test.metrostotales <= 805)), 'metrostotales_agrupada_800'] = 1

In [15]:
test['fecha'] = pd.to_datetime(test['fecha'], infer_datetime_format=True)
test['mes'] = test['fecha'].dt.month
test['dia'] = test['fecha'].dt.dayofweek
test['anio'] = test['fecha'].dt.year

In [16]:
test.loc[test.tipodepropiedad.str.contains('Comercial', na=False), 'tipodepropiedad'] = 'Comercio'
test.loc[test.tipodepropiedad.str.contains('Casa en condominio', na=False), 'tipodepropiedad'] = 'Condominio'

In [17]:
## LIMPIAMOS LOS VALORES NULOS O NAN QUE NO NOS SIRVEN. (CRITERIO)

train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [18]:
## COLUMNAS DONDE POR CADA REGISTRO SABEMOS QUE TENEMOS UN VALOR (SIEMPRE PRESENTES)

train['provincia'] = train['provincia'].astype('category')
dummies = pd.get_dummies(train['provincia'], drop_first=False)
train = pd.concat([train, dummies], axis=1)
dummies = ''

train['tipodepropiedad'] = train['tipodepropiedad'].astype('category')
dummies = pd.get_dummies(train['tipodepropiedad'], drop_first=False)
train = pd.concat([train, dummies], axis=1)
dummies = ''

train['anio'] = train['anio'].astype('category')
dummies = pd.get_dummies(train['anio'], drop_first=False)
train = pd.concat([train, dummies], axis=1)
dummies = ''

In [19]:
## COLUMNAS DONDE POR CADA REGISTRO SABEMOS QUE TENEMOS UN VALOR (SIEMPRE PRESENTES)

test['provincia'] = test['provincia'].astype('category')
dummies = pd.get_dummies(test['provincia'], drop_first=False)
test = pd.concat([test, dummies], axis=1)
dummies = ''

test['tipodepropiedad'] = test['tipodepropiedad'].astype('category')
dummies = pd.get_dummies(test['tipodepropiedad'], drop_first=False)
test = pd.concat([test, dummies], axis=1)
dummies = ''

test['anio'] = test['anio'].astype('category')
dummies = pd.get_dummies(test['anio'], drop_first=False)
test = pd.concat([test, dummies], axis=1)
dummies = ''

### =====================================================================
### LIMPIEZA Y COLUMNAS RESULTANTES.
### =====================================================================

In [20]:
## ELIMINAMOS LAS COLUMNAS QUE NO SON NUMÉRICAS.
## PERO TENERLAS PRESENTES PARA HACER UN TRABAJO POSTERIOR.

train.drop('titulo', axis=1, inplace=True)
train.drop('descripcion', axis=1, inplace=True)
train.drop('tipodepropiedad', axis=1, inplace=True)
train.drop('direccion', axis=1, inplace=True)
train.drop('ciudad', axis=1, inplace=True)
train.drop('provincia', axis=1, inplace=True)
train.drop('fecha', axis=1, inplace=True)

test.drop('titulo', axis=1, inplace=True)
test.drop('descripcion', axis=1, inplace=True)
test.drop('tipodepropiedad', axis=1, inplace=True)
test.drop('direccion', axis=1, inplace=True)
test.drop('ciudad', axis=1, inplace=True)
test.drop('provincia', axis=1, inplace=True)
test.drop('fecha', axis=1, inplace=True)

In [21]:
## ELIMINAMOS COLUMNAS NUMÉRICAS QUE CARECEN DE SENTIDO.

train.drop('idzona', axis=1, inplace=True)
train.drop('lat', axis=1, inplace=True)
train.drop('lng', axis=1, inplace=True)

test.drop('idzona', axis=1, inplace=True)
test.drop('lat', axis=1, inplace=True)
test.drop('lng', axis=1, inplace=True)

train.drop('dia', axis=1, inplace=True)
train.drop('mes', axis=1, inplace=True)
#train.drop('anio', axis=1, inplace=True)

test.drop('dia', axis=1, inplace=True)
test.drop('mes', axis=1, inplace=True)
#test.drop('anio', axis=1, inplace=True)

In [22]:
test['hospedaje'] = 0
test['garage'] = 0

In [23]:
train.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 149 columns):
id                               int64
antiguedad                       float64
habitaciones                     float64
garages                          float64
banos                            float64
metroscubiertos                  float64
metrostotales                    float64
gimnasio                         float64
usosmultiples                    float64
piscina                          float64
escuelascercanas                 float64
centroscomercialescercanos       float64
precio                           float64
excelente                        int64
vigilancia                       int64
esquina                          int64
remodelada                       int64
lujo                             int64
antiguedad_agrupada_0            int64
antiguedad_agrupada_10           int64
antiguedad_agrupada_20           int64
antiguedad_agrupada_30           int64
antigu

In [24]:
test.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 148 columns):
id                               int64
antiguedad                       float64
habitaciones                     float64
garages                          float64
banos                            float64
metroscubiertos                  float64
metrostotales                    float64
gimnasio                         float64
usosmultiples                    float64
piscina                          float64
escuelascercanas                 float64
centroscomercialescercanos       float64
excelente                        int64
vigilancia                       int64
esquina                          int64
remodelada                       int64
lujo                             int64
antiguedad_agrupada_0            int64
antiguedad_agrupada_10           int64
antiguedad_agrupada_20           int64
antiguedad_agrupada_30           int64
antiguedad_agrupada_40           int64
antiguedad

In [25]:
## eventos['mesMayus'] = eventos['mesMayus'].astype('category')

## COLUMNAS DONDE POR CADA REGISTRO SABEMOS QUE TENEMOS UN VALOR (SIEMPRE PRESENTES)
## dummies = pd.get_dummies(eventos['diasemana'], drop_first=False)
## eventos = pd.concat([eventos, dummies], axis=1)
## dummies = ''

In [26]:
train.to_csv('DATA/train.csv', index=False)
test.to_csv('DATA/test.csv', index=False)